In [ ]:
import os
import keras
import seaborn
import pandas as pd

from tqdm import tqdm
from keras.models import load_model
import tensorflow as tf

from tensorflow.keras.preprocessing.image import load_img, img_to_array

import numpy as np
import scipy
from numpy import expand_dims

from keras.applications.vgg19 import preprocess_input

import matplotlib.pyplot as plt
from pathlib import Path
import pathlib

## Importation des dessins

In [ ]:
# importation des données
os.chdir(os.path.expanduser("/home/jfalck/PycharmProjects/drawinganalyses"))
LOCAL_DATA_DIR = Path("/home/jfalck/PycharmProjects/drawinganalyses/drawinganalyses/data/Humans")
# Dossier de stockages des modeles
MODEL_DATA_DIR = Path("/home/jfalck/PycharmProjects/drawinganalyses/drawinganalyses/models/Humans")

categories = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2','NOV','EXP']
#categories.remove('.ipynb_checkpoints')
drawings = []
labels = []
indices = []


for folder in os.listdir(LOCAL_DATA_DIR):
    if os.path.isdir(os.path.join(LOCAL_DATA_DIR, folder)):
        if folder in categories:
            for img in os.listdir(os.path.join(LOCAL_DATA_DIR, folder)):
                if not img.startswith('.'):
                    labels.append(folder)
                    indices.append(os.path.join(folder, img))
                    img = load_img(os.path.join(LOCAL_DATA_DIR,folder, img), target_size=(224,224,3))
                    img = img_to_array(img)
                    img = expand_dims(img,axis=0)
                    img = preprocess_input(img)
                    img = np.array(img)
                    img = np.reshape(img, (1,224,224,3))
                    drawings.append(img)


In [ ]:
# Pour chaque modèle couple VS NOV+EXP, on regarde comment sont classifiés les autres groupes

# liste de toutes les catégories (sauf adultes)
categories = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2']

for i in tqdm(range(len(categories)-1)):
    categories = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2']
    categories_model = categories[i:(i+2)]
    model = load_model(os.path.join(MODEL_DATA_DIR,
                       str(categories_model[0] + categories_model[1]),
                       str(categories_model[0] + categories_model[1]) + '.h5'))

In [ ]:
list(range(len(categories)-1))

In [ ]:
categories = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2']
# Vérifier que pour chaque modèle, valeur proche de 0 = enfants et 1 = adultes
# Pour un modèle (par ex CrèchePS vs adultes), on donne tous les dessins CrèchePS et on regarde si les valeurs prédites sont petites
for i in tqdm(range(len(categories)-1)):
    plt.subplot(3, 3, (i+1)) 
    categories = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2']
    categories_model = categories[i:(i+2)]
    categories_to_predict = categories_model # on veut préd
    predictions = []
    predictions_true_label = []
    pred_names = []
    pred_dict = {category:[] for category in categories_to_predict}
    mean_pred = []
    sd_pred = []
    for category in categories_to_predict: # pour les 2 catégories enfants sur lesquels le modèle a été entrainé
        tmp_indices = np.where(np.array(labels)==category)[0] # on extrait les indices des dessins de ces catégories
        for idx in tmp_indices: # pour chaque dessin de cette catégorie
            # on prédit le groupe
            model = load_model(os.path.join(MODEL_DATA_DIR,
                       str(categories_model[0] + categories_model[1]),
                       str(categories_model[0] + categories_model[1]) + '.h5'))
            tmp_pred = model.predict(drawings[idx])
            pred_dict[category].append(tmp_pred)
            # on rajoute la valeur de la prédiction dans une autre liste, ainsi que le vrai label, et le nom (pour retrouver le dessinateur)
            predictions.append(tmp_pred[0][0]) # on rajoute la valeur de la prédiction pour ce dessin
            predictions_true_label.append(category) # on rajoute le vrai label de ce dessin
    plt.title(categories_to_predict)
    plt.hist(predictions)

In [ ]:
# Vérifier que pour chaque modèle, valeur proche de 0 = enfants et 1 = adultes
# Pour un modèle (par ex CrèchePS vs adultes), on donne tous les dessins CrèchePS et on regarde si les valeurs prédites sont petites
for i in tqdm(range(len(['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2'])-1)):
    plt.subplot(3, 3, (i+1)) 
    categories = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2','NOV','EXP']
    categories_model = categories[i:(i+2)]
    categories_to_predict = ['NOV','EXP'] # on veut préd
    predictions = []
    predictions_true_label = []
    pred_names = []
    pred_dict = {category:[] for category in categories_to_predict}
    mean_pred = []
    sd_pred = []
    for category in categories_to_predict: # pour les 2 catégories enfants sur lesquels le modèle a été entrainé
        tmp_indices = np.where(np.array(labels)==category)[0] # on extrait les indices des dessins de ces catégories
        for idx in tmp_indices: # pour chaque dessin de cette catégorie
            # on prédit le groupe
            model = load_model(os.path.join(MODEL_DATA_DIR,
                       str(categories_model[0] + categories_model[1]),
                       str(categories_model[0] + categories_model[1]) + '.h5'))
            tmp_pred = model.predict(drawings[idx])
            pred_dict[category].append(tmp_pred)
            # on rajoute la valeur de la prédiction dans une autre liste, ainsi que le vrai label, et le nom (pour retrouver le dessinateur)
            predictions.append(tmp_pred[0][0]) # on rajoute la valeur de la prédiction pour ce dessin
            predictions_true_label.append(category) # on rajoute le vrai label de ce dessin
    plt.title(categories_model)
    plt.hist(predictions)

In [ ]:
#CrèchePS : adultes = 1
#PSMS : adultes = 0
#MSGS : adultes = 1
#GSCE1 : adultes = 1
#CE1CE2 : adultes = 1
#CE2CM1 : adultes = 1
#CM1CM2 : adultes = 1

In [ ]:
# Pour chaque modèle couple VS NOV+EXP, on regarde comment sont classifiés les autres groupes

# liste de toutes les catégories (sauf adultes)
categories = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2']

# création du dataframe qui contiendra les résultats
categories_couple = []
for j in range(len(categories)-1):
    categories_couple.append(categories[j]+categories[j+1])
matrix_pred = np.full((len(categories)-1,len(categories)),np.NaN)
matrix_pred = pd.DataFrame(matrix_pred, columns=categories, index=categories_couple)
print(matrix_pred)

for i in tqdm(range(len(categories)-1)):
    categories = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2']
    categories_model = categories[i:(i+2)]
    del categories[i:(i+2)]
    categories_to_predict = categories

    predictions = []
    predictions_true_label = []
    pred_names = []
    pred_dict = {category:[] for category in categories_to_predict}
    mean_pred = []
    sd_pred = []

    model = load_model((os.path.join(MODEL_DATA_DIR,
                       str(categories_model[0] + categories_model[1]),
                       str(categories_model[0] + categories_model[1]) + '.h5')))
    for category in categories_to_predict: # pour chaque catégorie
        tmp_indices = np.where(np.array(labels)==category)[0]
        for idx in tmp_indices: # pour chaque dessin de cette catégorie
            # on prédit le groupe
            tmp_pred = model.predict(drawings[idx])
            # on rajoute la valeur de la prédiction dans le dictionnaire
            pred_dict[category].append(tmp_pred)
            # on rajoute la valeur de la prédiction dans une autre liste, ainsi que le vrai label, et le nom (pour retrouver le dessinateur)
            predictions.append(tmp_pred[0][0]) # on rajoute la valeur de la prédiction pour ce dessin
            predictions_true_label.append(category) # on rajoute le vrai label de ce dessin
            # on prend le chemin du dessin pour extraire le nom du dessinateur
            path_name = indices[idx]
            pred_names.append(path_name.split('/')[1].split('.')[0])
        mean_pred.append(np.mean(pred_dict[category]))
        sd_pred.append(np.std(pred_dict[category]))
        k=0
        for categorie_to_predict in categories_to_predict:
            print(matrix_pred.head(5))
            print("Cat: " , category)
            print("Cat_predi : " , categorie_to_predict)
            print("indice col : ", matrix_pred.columns.get_loc(categorie_to_predict))
            print("indice: ", i)
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
            print(mean_pred)
            if k>=len(mean_pred):
                matrix_pred.iloc[i, matrix_pred.columns.get_loc(categorie_to_predict)] = 0
            else:
                matrix_pred.iloc[i, matrix_pred.columns.get_loc(categorie_to_predict)] = mean_pred[k]
            k+=1

In [ ]:
matrix_pred.iloc[1] = 1-matrix_pred.iloc[1]
# comme le modèle PSMS vs adultes est le seul pour lequel adultes=0 et enfants=1, on fait 1-pred pour avoir adultes=1 et enfants=0

In [ ]:
seaborn.heatmap(matrix_pred,annot=True)

In [ ]:
import seaborn as sns

#corr = df2.corr() #your dataframe

# figsize=(6, 6) control width and height
# dpi = 600, I 
plt.figure(figsize=(6, 6), 
           dpi = 600) 
 
# parameter annot_kws={"size": 8} control corr values font size
sns.heatmap(matrix_pred, cmap="Blues", annot=True, annot_kws={"size": 8})

In [ ]:
plt.errorbar(x=np.arange(len(categories_to_predict)), y=mean_pred, yerr=sd_pred)
plt.xticks(np.arange(len(categories_to_predict)),categories_to_predict)
#plt.yticks(np.arange(2),['CrèchePS','NOVEXP'])
plt.title('Predictions of drawings depending \non the age category', pad = 15)
plt.xlabel('Age category', labelpad = 10)
plt.ylabel('Mean prediction value', labelpad = -40)
plt.show()

In [ ]:
pred_accuracy = {}
sorted_categories = ['Chimpanzés', 'MS', 'GS', 'CE1', 'CE2', 'CM1', 'CM2']
for sorted_category in sorted_categories:
    np.where(np.array(predictions_true_label) == sorted_category)[0]